In [5]:
import tkinter as tk
from tkinter import ttk
import pandas as pd

df = pd.read_excel("tabela_exemplo.xlsx")
df["Salário"] = df["Salário"].astype(float)

colunas = ["Cidade","Profissão","Nome","Idade","Salário"]

janela = tk.Tk()
janela.title("Tabela Interativa com Agrupamento de Colunas")
janela.configure(bg="white")

style = ttk.Style()
style.configure("Cidade.Treeview",
                background="yellow",
                foreground="black")

titulo = tk.Label(janela,
                  text="Projeto Tabela Interativa com Filtros",
                  font=("Arial",16),
                  bg="white")
titulo.grid(row=0,column=0,columnspan=3,pady=(10,10))

container_filtros = tk.Frame(janela,bg="white")
container_filtros.grid(row=1,column=0,columnspan=3,pady=(10,10))

filtros = {}

for idx,col in enumerate(colunas):
    lbl = tk.Label(container_filtros,
                   text=col,
                   font=("Arial",12),
                   bg="white")
    lbl.grid(row=0,column=idx,padx=(5,5))

    filtro = tk.Entry(container_filtros,
                      font=("Arial",12))
    filtro.grid(row=1,column=idx,padx=(5,5))
    filtros[col] = filtro

tree = ttk.Treeview(janela,columns=colunas,show="tree headings")
tree.heading("#0",text="Cidade")

for col in colunas:
    tree.heading(col,text=col)

tree.grid(row=3,column=0, columnspan=3,pady=10)

def formatar_salario(valor):
    formatted = f"{valor:,.2f}"
    return formatted.replace(",","X").replace(".",",").replace("X",".")

def carregar_dados():
    tree.delete(*tree.get_children())

    df_grouped = df.groupby('Cidade', as_index=False).sum()

    for _, row in df_grouped.iterrows():
        cidade = row['Cidade']

        salario_atual = formatar_salario(row['Salário'])

        cidade_id = tree.insert("",
                                tk.END,
                                text=cidade,
                                values=["","","","",salario_atual],
                                open=False,
                                tags=("Cidade",))
        registros_cidade = df[df['Cidade'] == cidade]

        for _,detalhe in registros_cidade.iterrows():
            valores = list(detalhe)
            valores[4] = formatar_salario(valores[4])
            valores[0] = ""
            
            tree.insert(cidade_id,tk.END,values=valores)
            tree.tag_configure("Cidade",background="yellow")
            
            atualizar_visibilidade_colunas()

def filtrar_dados(*args):
    tree.delete(*tree.get_children())

    df_filtrado = df.copy()
    
    for col,entry in filtros.items():
        filtro_valor = entry.get().strip().lower()

        if filtro_valor:
            df_filtrado = df_filtrado[df_filtrado[col].astype(str).str.lower().str.contains(filtro_valor)]

    df_grouped = df_filtrado.groupby('Cidade', as_index=False).sum()

    for _, linha in df_grouped.iterrows():
        cidade = linha['Cidade']

        salario_total = formatar_salario(linha['Salário'])

        cidade_id = tree.insert('',tk.END,
                                text=cidade,
                                values=["","","","",salario_total],
                                open=False,
                                tags=("Cidade",))
        
        registros_cidade = df_filtrado[df_filtrado['Cidade']==cidade]

        for _,detalhe in registros_cidade.iterrows():
            valores = list(detalhe)
            valores[4] = formatar_salario(valores[4])
            valores[0] = ""

            tree.insert(cidade_id,tk.END,values=valores)

        tree.tag_configure("Cidade",background="yellow")

        atualizar_visibilidade_colunas()

def atualizar_visibilidade_colunas():
    if colunas_ocultas:
        tree["displaycolumns"] = colunas[1:]
        btn_agrupar_colunas.config(text='-')
    else:
        tree["displaycolumns"] = ("Salário",)
        btn_agrupar_colunas.config(text='+')

def toggle_colunas():
    global colunas_ocultas

    colunas_ocultas = not colunas_ocultas

    atualizar_visibilidade_colunas()

colunas_ocultas = False

btn_agrupar_colunas = tk.Button(janela,
                                text="+",
                                command=toggle_colunas,
                                font=("Arial",12),
                                width=3)
btn_agrupar_colunas.grid(row=2,column=2,pady=(5,10))

janela.grid_columnconfigure(0,weight=1)
janela.grid_rowconfigure(3,weight=1)

for entry in filtros.values():
    entry.bind("<KeyRelease>",filtrar_dados)

carregar_dados()

janela.mainloop()